In [1]:
from os.path import join
import argparse
import torch
import torch.nn as nn
import time
import math
from torch.autograd import Variable
from PIL import Image
import import_ipynb
from config import DEVICE
from torchvision.transforms import ToTensor
import numpy as np
from model import VDSR

importing Jupyter notebook from config.ipynb
importing Jupyter notebook from model.ipynb


In [2]:
img = Image.open('test_scale2x.jpg').convert('YCbCr')
y, cb, cr = img.split()
scale_factor = 2

In [19]:
img = img.resize((int(img.size[0]*scale_factor),
                  int(img.size[1]*scale_factor)), Image.BICUBIC)
img.save('yee.jpg')

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  


In [6]:
vdsr = VDSR().to(DEVICE)
checkpoint = torch.load('./output/model_epoch_25.pth')
vdsr.load_state_dict(checkpoint)

<All keys matched successfully>

In [8]:
input = Variable(ToTensor()(img)).view(1, -1, img.size[1], img.size[0])
input = input.cuda()

In [13]:
out = vdsr(input)
out = out.to('cpu')

In [15]:
print("type = ", type(out))
out_img_y = out.data[0].numpy()
out_img_y *= 255.0
out_img_y = out_img_y.clip(0, 255)
out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

type =  <class 'torch.Tensor'>


In [16]:
out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
out_img = Image.merge(
    'YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  """Entry point for launching an IPython kernel.
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  


In [18]:
out_img.save('output.jpg')